In [26]:
import pandas as pd
from datetime import date
import calendar

In [38]:
meteo = pd.DataFrame()
meteorological_var = ['fecha','hora','temperatura','humedad','presion_atm','dd','viento','nombre']

print('Obteniendo datos del SMN. Fuente: https://www.smn.gob.ar/descarga-de-datos')
for mes in range(12):
    mes = mes +1
    dias = calendar.monthrange(2023, mes)[1]

    if mes < 10:
        aamm = '20230'+str(mes)
    else:
        aamm = '2023'+str(mes)
        
    for dia in range(dias):
        dia = dia +1
        if dia < 10:
            fecha = aamm +'0'+str(dia)
        else:
            fecha = aamm + str(dia)

        print('Fecha:',fecha)
        try:
            csv = 'https://ssl.smn.gob.ar/dpd/descarga_opendata.php?file=observaciones/datohorario' + fecha + '.txt'
            df = pd.read_fwf(csv, colspecs='infer', decimal='.', skiprows=2, encoding ='latin1', names=meteorological_var)
            df = df.loc[df['nombre']=='CORDOBA OBSERVATO'][['hora','temperatura','humedad','presion_atm','dd','viento']]
            df['fecha'] = fecha
            meteo = pd.concat([meteo, df], ignore_index=True)
        except:
            print('Sin datos el ', fecha)
            df['fecha'] = fecha  #repito datos del día anterior
            meteo = pd.concat([meteo, df], ignore_index=True)
        

    

Obteniendo datos del SMN. Fuente: https://www.smn.gob.ar/descarga-de-datos
Fecha: 20230101
Fecha: 20230102
Fecha: 20230103
Fecha: 20230104
Fecha: 20230105
Fecha: 20230106
Fecha: 20230107
Fecha: 20230108
Fecha: 20230109
Fecha: 20230110
Fecha: 20230111
Fecha: 20230112
Fecha: 20230113
Fecha: 20230114
Fecha: 20230115
Fecha: 20230116
Fecha: 20230117
Fecha: 20230118
Fecha: 20230119
Fecha: 20230120
Fecha: 20230121
Fecha: 20230122
Fecha: 20230123
Fecha: 20230124
Fecha: 20230125
Fecha: 20230126
Fecha: 20230127
Fecha: 20230128
Fecha: 20230129
Fecha: 20230130
Fecha: 20230131
Fecha: 20230201
Fecha: 20230202
Fecha: 20230203
Fecha: 20230204
Fecha: 20230205
Fecha: 20230206
Fecha: 20230207
Fecha: 20230208
Fecha: 20230209
Fecha: 20230210
Fecha: 20230211
Fecha: 20230212
Fecha: 20230213
Fecha: 20230214
Fecha: 20230215
Fecha: 20230216
Fecha: 20230217
Fecha: 20230218
Fecha: 20230219
Fecha: 20230220
Fecha: 20230221
Fecha: 20230222
Fecha: 20230223
Fecha: 20230224
Fecha: 20230225
Fecha: 20230226
Fecha: 202302

Descarga del Catálogo de Datos Abiertos del SMN: https://www.smn.gob.ar/descarga-de-datos 
* fecha: AAAAMMDD
* hora: hh
* temperatura: grados centígrados
* humedad: %
* presion_atm: (presión atmosférica al nivel del mar) en hPa
* dd: en gramos?
* viento: (velicidad) en km/hr

https://www.meted.ucar.edu/glossaries/mesogloss_es.htm

In [39]:
meteo

,hora,temperatura,humedad,presion_atm,dd,viento,fecha
0,0.0,23.0,85.0,1007.6,320.0,7.0,20230101
1,1.0,22.0,92.0,1007.5,180.0,6.0,20230101
2,2.0,21.4,92.0,1008.8,150.0,6.0,20230101
3,3.0,20.8,92.0,1008.2,200.0,7.0,20230101
4,4.0,20.6,93.0,1007.6,180.0,7.0,20230101
...,...,...,...,...,...,...,...
8706,19.0,30.9,38.0,1006.4,50.0,13.0,20231231
8707,20.0,28.5,52.0,1007.7,50.0,15.0,20231231
8708,21.0,26.8,62.0,1009.3,90.0,17.0,20231231
8709,22.0,25.7,61.0,1009.9,90.0,22.0,20231231


In [40]:
# Convert objects to datetime fields
meteo['fecha'] = pd.to_datetime(meteo['fecha'], format='%Y%m%d')

In [43]:
# Cálculo de Wind chill o sensación térmica por frio
# https://www.tiempo.com/ram/289402/la-sensacion-termica-y-sus-calculos-por-la-aemet/
meteo['sensacion_termica_invierno']= 13.12 + 0.6215 * meteo['temperatura'] - 11.37 * meteo['viento']**0.16 + 0.3965 * meteo['viento']**0.16


In [44]:
# Sensación Térmica por Calor o bochorno
# https://www.tiempo.com/ram/289402/la-sensacion-termica-y-sus-calculos-por-la-aemet/
meteo['sensacion_termica_verano']=  -8.78469476 + 1.61139411 * meteo['temperatura'] + 2.338548839 * meteo['humedad'] - 0.14611605 * meteo['temperatura'] * meteo['humedad'] - 0.012308094 * meteo['temperatura']**2 - 0.01642828 * meteo['humedad']**2 + 0.002211732 * meteo['temperatura']**2 * meteo['humedad'] + 0.00072546 * meteo['temperatura'] * meteo['humedad']**2 + 0.000003582 * meteo['temperatura']**2 * meteo['humedad']**2    

In [46]:
# Sensación térmica por frio solo válida en el rango +10 C y -50 ºC. 
meteo['sensacion_termica'] = meteo.loc[meteo['temperatura']<=10]['sensacion_termica_invierno']
# Sensación térmica por calor solo válida en el rango de temp superiores a 26 ºC y humedades superiores al 40%
meteo['sensacion_termica'] = meteo.loc[(meteo['temperatura']>=26) & (meteo['humedad']>=40)]['sensacion_termica_verano']
# Para el resto la sensación térmica debería ser igual a la temperatura atmosferica
meteo['sensacion_termica'] = meteo['sensacion_termica'].fillna(meteo['temperatura'])

In [47]:
meteo

,hora,temperatura,humedad,presion_atm,dd,viento,fecha,sensacion_termica_invierno,sensacion_termica_verano,sensacion_termica
0,0.0,23.0,85.0,1007.6,320.0,7.0,2023-01-01,12.432808,49.886177,23.000000
1,1.0,22.0,92.0,1007.5,180.0,6.0,2023-01-01,12.176299,49.311898,22.000000
2,2.0,21.4,92.0,1008.8,150.0,6.0,2023-01-01,11.803399,46.958910,21.400000
3,3.0,20.8,92.0,1008.2,200.0,7.0,2023-01-01,11.065508,44.765394,20.800000
4,4.0,20.6,93.0,1007.6,180.0,7.0,2023-01-01,10.941208,44.343496,20.600000
...,...,...,...,...,...,...,...,...,...,...
8706,19.0,30.9,38.0,1006.4,50.0,13.0,2023-12-31,15.782804,40.384633,30.900000
8707,20.0,28.5,52.0,1007.7,50.0,15.0,2023-12-31,13.908097,44.972279,44.972279
8708,21.0,26.8,62.0,1009.3,90.0,17.0,2023-12-31,12.509196,47.730070,47.730070
8709,22.0,25.7,61.0,1009.9,90.0,22.0,2023-12-31,11.098339,44.243861,25.700000


In [48]:
# Se elimino columnas innecesarias 
meteo.drop(['sensacion_termica_verano','sensacion_termica_invierno'], axis=1, inplace=True)

In [49]:
meteo

,hora,temperatura,humedad,presion_atm,dd,viento,fecha,sensacion_termica
0,0.0,23.0,85.0,1007.6,320.0,7.0,2023-01-01,23.000000
1,1.0,22.0,92.0,1007.5,180.0,6.0,2023-01-01,22.000000
2,2.0,21.4,92.0,1008.8,150.0,6.0,2023-01-01,21.400000
3,3.0,20.8,92.0,1008.2,200.0,7.0,2023-01-01,20.800000
4,4.0,20.6,93.0,1007.6,180.0,7.0,2023-01-01,20.600000
...,...,...,...,...,...,...,...,...
8706,19.0,30.9,38.0,1006.4,50.0,13.0,2023-12-31,30.900000
8707,20.0,28.5,52.0,1007.7,50.0,15.0,2023-12-31,44.972279
8708,21.0,26.8,62.0,1009.3,90.0,17.0,2023-12-31,47.730070
8709,22.0,25.7,61.0,1009.9,90.0,22.0,2023-12-31,25.700000


Como no tenemos consumos por rango horario si no por franjas:
* resto = (5am a 6pm)  = 13 hs
* pico (6pm a 11pm) = 5 hs 
* valle (11pm a 5am) = 6 hs

Se computan las variables metereológicas para esas franjas con los valores promedios.

In [50]:
meteo_resto = meteo.loc[(meteo['hora']>=5) &(meteo['hora']<18)].groupby('fecha')[['temperatura','humedad','presion_atm','dd','viento','sensacion_termica']].mean()
meteo_resto['banda'] = 'resto'
meteo_pico = meteo.loc[(meteo['hora']>=18) &(meteo['hora']<23)].groupby('fecha')[['temperatura','humedad','presion_atm','dd','viento','sensacion_termica']].mean()
meteo_pico['banda'] = 'pico'
meteo_valle = meteo.loc[(meteo['hora']>=23) | (meteo['hora']<5)].groupby('fecha')[['temperatura','humedad','presion_atm','dd','viento','sensacion_termica']].mean()
meteo_valle['banda'] = 'valle'

In [52]:
meteorologia = pd.concat([meteo_resto, meteo_pico])
meteorologia = pd.concat([meteo_valle, meteorologia])

In [53]:
meteorologia.sort_values('fecha')

,temperatura,humedad,presion_atm,dd,viento,sensacion_termica,banda
fecha,,,,,,,
2023-01-01,21.266667,87.000000,1009.083333,210.000000,7.333333,21.266667,valle
2023-01-01,24.080000,52.000000,1010.960000,184.000000,15.600000,25.960202,pico
2023-01-01,23.146154,77.846154,1009.046154,183.846154,10.230769,27.730872,resto
2023-01-02,18.566667,69.166667,1013.250000,255.000000,4.666667,18.566667,valle
2023-01-02,28.960000,28.600000,1006.700000,88.000000,13.000000,28.960000,pico
...,...,...,...,...,...,...,...
2023-12-30,23.061538,54.153846,1017.461538,78.461538,9.538462,25.930408,resto
2023-12-30,22.400000,43.333333,1016.866667,170.000000,7.333333,22.400000,valle
2023-12-31,22.300000,71.333333,1012.700000,65.000000,10.166667,22.300000,valle


In [56]:
meteorologia.shape

(1089, 7)

In [55]:
# nombre de archivo
csv_file = 'meteorologia_cordoba.csv'

# grabar los datos
meteorologia.to_csv(csv_file, sep=';')